## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,77.47,80,67,8.32,broken clouds
1,1,Albany,US,42.6001,-73.9662,71.76,48,67,1.01,broken clouds
2,2,Rikitea,PF,-23.1203,-134.9692,80.40,77,46,13.33,scattered clouds
3,3,La Ronge,CA,55.1001,-105.2842,46.65,34,20,8.05,few clouds
4,4,Luderitz,NaN,-26.6481,15.1594,71.11,60,63,6.44,broken clouds


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 90


In [40]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.


preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,77.47,80,67,8.32,broken clouds
1,1,Albany,US,42.6001,-73.9662,71.76,48,67,1.01,broken clouds
2,2,Rikitea,PF,-23.1203,-134.9692,80.40,77,46,13.33,scattered clouds
4,4,Luderitz,NaN,-26.6481,15.1594,71.11,60,63,6.44,broken clouds
5,5,Salalah,OM,17.0151,54.0924,75.42,23,0,16.13,clear sky
6,6,Butaritari,KI,3.0707,172.7902,80.53,81,23,19.15,light rain
7,7,Geraldton,AU,-28.7667,114.6000,70.21,83,95,10.36,overcast clouds
8,8,Saldanha,ZA,-33.0117,17.9442,75.25,83,6,10.74,clear sky
9,9,Lorengau,PG,-2.0226,147.2712,82.56,75,48,7.45,scattered clouds
10,10,Peruibe,BR,-24.3200,-46.9983,80.96,90,83,4.92,broken clouds


In [41]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                20
City                   20
Country                19
Lat                    20
Lng                    20
Max Temp               20
Humidity               20
Cloudiness             20
Wind Speed             20
Current Description    20
dtype: int64

In [42]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')


In [23]:
clean_df.count()

City_ID                19
City                   19
Country                19
Lat                    19
Lng                    19
Max Temp               19
Humidity               19
Cloudiness             19
Wind Speed             19
Current Description    19
dtype: int64

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cidreira,BR,77.47,broken clouds,-30.1811,-50.2056,
1,Albany,US,71.76,broken clouds,42.6001,-73.9662,
2,Rikitea,PF,80.40,scattered clouds,-23.1203,-134.9692,
5,Salalah,OM,75.42,clear sky,17.0151,54.0924,
6,Butaritari,KI,80.53,light rain,3.0707,172.7902,
7,Geraldton,AU,70.21,overcast clouds,-28.7667,114.6000,
8,Saldanha,ZA,75.25,clear sky,-33.0117,17.9442,
9,Lorengau,PG,82.56,scattered clouds,-2.0226,147.2712,
10,Peruibe,BR,80.96,broken clouds,-24.3200,-46.9983,
12,Makakilo City,US,80.01,broken clouds,21.3469,-158.0858,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
  
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
   
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [59]:
hotel_df.count()

City                   19
Country                19
Max Temp               19
Current Description    19
Lat                    19
Lng                    19
Hotel Name             19
dtype: int64

In [61]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df.head()
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cidreira,BR,77.47,broken clouds,-30.1811,-50.2056,Hotel Castelo
1,Albany,US,71.76,broken clouds,42.6001,-73.9662,
2,Rikitea,PF,80.40,scattered clouds,-23.1203,-134.9692,People ThankYou
5,Salalah,OM,75.42,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
6,Butaritari,KI,80.53,light rain,3.0707,172.7902,Isles Sunset Lodge


In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))